In [27]:
import time
import random
import logging
from collections import deque
from urllib.parse import urljoin, urlparse

import requests
from bs4 import BeautifulSoup, Comment

# ==== 設定 ====
ROOT = "https://www.musashino-u.ac.jp/"
HEADERS = {"User-Agent": "Mozilla/5.0 (compatible; SimpleSiteMap/1.0)"}
# 除外したい拡張子（小文字でチェック）
EXCLUDE_EXT = {
    ".pdf", ".jpg", ".jpeg", ".png", ".gif",
    ".zip", ".xlsx", ".xls", ".doc", ".docx",
    ".ppt", ".pptx", ".mp4", ".mov", ".css", ".js", ".ico",
}

# 最大取得ページ数（動作確認用に上限を設けておくと安全）
MAX_PAGES = 200

# ロギング
logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")


def is_internal(url: str, root: str) -> bool:
    """rootドメイン配下かどうかを判定する簡易チェック"""
    if not url:
        return False
    return url.startswith(root)


def has_bad_ext(url: str) -> bool:
    """URLが除外対象の拡張子で終わるかどうかをチェック"""
    lower = url.lower().split("?")[0]  # クエリ削る
    return any(lower.endswith(ext) for ext in EXCLUDE_EXT)


def fetch_page(session: requests.Session, url: str, timeout: float = 10.0):
    """HTTP GETしてBeautifulSoupを返す（失敗時はNone）"""
    try:
        resp = session.get(url, timeout=timeout)
        resp.encoding = resp.apparent_encoding
        return BeautifulSoup(resp.text, "html.parser")
    except requests.RequestException as e:
        logging.warning("取得失敗: %s (%s)", url, e)
        return None


def strip_comments(soup: BeautifulSoup):
    """HTMLコメントを取り除く（コメント内のリンクを無視するため）"""
    for c in soup.find_all(string=lambda t: isinstance(t, Comment)):
        c.extract()


def collect_links(soup: BeautifulSoup, base_url: str):
    """ページ内の<a href>から絶対URLを生成して返す"""
    links = set()
    for a in soup.find_all("a", href=True):
        raw = a.get("href")
        if not raw:
            continue
        # 相対パスを絶対に
        joined = urljoin(base_url, raw)
        # 同ページアンカーを取り除く
        cleaned = joined.split("#")[0]
        links.add(cleaned)
    return links


def crawl(root_url: str, max_pages: int = MAX_PAGES):
    """幅優先で巡回し、{url: title} の辞書を返す"""
    session = requests.Session()
    session.headers.update(HEADERS)

    queue = deque([root_url])
    seen = set()
    sitemap = {}
    pages_processed = 0

    root_parsed = urlparse(root_url)
    root_base = f"{root_parsed.scheme}://{root_parsed.netloc}/"

    while queue and pages_processed < max_pages:
        current = queue.popleft()

        if current in seen:
            continue
        seen.add(current)

        # 内部サイト以外は無視
        if not is_internal(current, root_base):
            logging.debug("外部ドメインスキップ: %s", current)
            continue

        # 拡張子でスキップ
        if has_bad_ext(current):
            logging.debug("拡張子スキップ: %s", current)
            continue

        # サーバー負荷軽減のためランダム短待ち
        time.sleep(0.4 + random.random() * 0.8)

        soup = fetch_page(session, current)
        if soup is None:
            continue

        # コメント削除（コメントアウトされたリンクを除外）
        strip_comments(soup)

        # タイトル抽出
        title = "タイトルなし"
        if soup.title and soup.title.string:
            title = soup.title.string.strip()

        sitemap[current] = title
        pages_processed += 1
        logging.info("[%d] %s → %s", pages_processed, title, current)

        # ページ内リンクを収集してキューに追加
        for link in collect_links(soup, current):
            # 内部・拡張子・既出チェック
            if not is_internal(link, root_base):
                continue
            if has_bad_ext(link):
                continue
            if link in seen or link in queue:
                continue
            queue.append(link)

    logging.info("完了：処理済みページ数=%d", pages_processed)
    return sitemap


if __name__ == "__main__":
    result = crawl(ROOT, max_pages=1000)
    print("\n=== サイトマップ（抜粋） ===")
    for i, (u, t) in enumerate(result.items()):
        if i >= 50:
            break
        print(f"{i+1:03d}: {t} -> {u}")

INFO: [1] 武蔵野大学 → https://www.musashino-u.ac.jp/
INFO: [2] 武蔵野INITIAL | 武蔵野大学 → https://www.musashino-u.ac.jp/basic/initial/
INFO: [3] 入試情報 | 武蔵野大学 → https://www.musashino-u.ac.jp/admission/
INFO: [4] 武蔵野大学で学びたい方 | 武蔵野大学 → https://www.musashino-u.ac.jp/prospective-students.html
INFO: [5] 本学公認クラブ「和太鼓 隼」と「武蔵野大学ウインドアンサンブル」がゆりかもめ30周年記念車両基地公開イベントに出演 | ニュース | 武蔵野大学 → https://www.musashino-u.ac.jp/news/detail/20251030-7383.html
INFO: [6] 第６回経営研究所シンポジウム「AI時代に創る未来のキャリア 起業とEXITのリアル」 | イベント | 武蔵野大学 → https://www.musashino-u.ac.jp/event/detail/20251112-7302.html
INFO: [7] このサイトについて | 武蔵野大学 → https://www.musashino-u.ac.jp/accesibility-and-copyright.html
INFO: [8] 保護者の方 | 武蔵野大学 → https://www.musashino-u.ac.jp/parents.html
INFO: [9] 連携研究の取り組み | 研究 | 武蔵野大学 → https://www.musashino-u.ac.jp/research/alignment/collaborativeresearch.html
INFO: [10] 学問の地平から | 武蔵野大学 → https://www.musashino-u.ac.jp/research/interview/
INFO: [11] 縁バースキャンパス | 武蔵野大学 → https://www.musashino-u.ac.jp/enverse/
INFO: [12] 武蔵野大学への留学 |


=== サイトマップ（抜粋） ===
001: 武蔵野大学 -> https://www.musashino-u.ac.jp/
002: 武蔵野INITIAL | 武蔵野大学 -> https://www.musashino-u.ac.jp/basic/initial/
003: 入試情報 | 武蔵野大学 -> https://www.musashino-u.ac.jp/admission/
004: 武蔵野大学で学びたい方 | 武蔵野大学 -> https://www.musashino-u.ac.jp/prospective-students.html
005: 本学公認クラブ「和太鼓 隼」と「武蔵野大学ウインドアンサンブル」がゆりかもめ30周年記念車両基地公開イベントに出演 | ニュース | 武蔵野大学 -> https://www.musashino-u.ac.jp/news/detail/20251030-7383.html
006: 第６回経営研究所シンポジウム「AI時代に創る未来のキャリア 起業とEXITのリアル」 | イベント | 武蔵野大学 -> https://www.musashino-u.ac.jp/event/detail/20251112-7302.html
007: このサイトについて | 武蔵野大学 -> https://www.musashino-u.ac.jp/accesibility-and-copyright.html
008: 保護者の方 | 武蔵野大学 -> https://www.musashino-u.ac.jp/parents.html
009: 連携研究の取り組み | 研究 | 武蔵野大学 -> https://www.musashino-u.ac.jp/research/alignment/collaborativeresearch.html
010: 学問の地平から | 武蔵野大学 -> https://www.musashino-u.ac.jp/research/interview/
011: 縁バースキャンパス | 武蔵野大学 -> https://www.musashino-u.ac.jp/enverse/
012: 武蔵野大学への留学 | 国際交流・留学 | 武蔵野大学 -> https://www.